In [75]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import requests
from datetime import datetime
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [62]:
retry_session = retry(requests.Session(), retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [63]:
today = datetime.now().strftime("%Y-%m-%d")
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [38.8951, 51.5085, 35.6895, 21.0245],
	"longitude": [-77.0364, -0.1257, 139.6917, 105.8412],
	"start_date": "2020-01-01",
	"end_date": today,
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"hourly": ["temperature_2m", "precipitation", "weather_code", "relative_humidity_2m", "wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

In [64]:
city = ["Washington", "London", "Tokyo", "HaNoi"]
all_data = []
for idx, response in enumerate(responses):
    Latitude = response.Latitude()
    Longitude = response.Longitude()
    Elevation = response.Elevation()
    data = {
        "City": city[idx],
        "Latitude": Latitude,
        "Longitude": Longitude,
        "Elevation": Elevation
    }    
    all_data.append(data)

df = pd.DataFrame(all_data)
df

,City,Latitude,Longitude,Elevation
0,Washington,38.910366,-77.072510,12.0
1,London,51.493847,-0.163025,23.0
2,Tokyo,35.676624,139.691116,40.0
3,HaNoi,21.054480,105.807106,10.0


In [65]:
all_hourly_data = []
for idx, response in enumerate(responses):
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
	hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}

	hourly_data["city"] = city[idx]
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["weather_code"] = hourly_weather_code
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

	df = pd.DataFrame(hourly_data)
	all_hourly_data.append(df)

hourly_df = pd.concat(all_hourly_data)
hourly_df.head()

,date,city,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
0,2020-01-01 00:00:00+00:00,Washington,7.7555,0.0,3.0,55.327991,9.726665
1,2020-01-01 01:00:00+00:00,Washington,7.5555,0.0,3.0,53.680984,8.699793
2,2020-01-01 02:00:00+00:00,Washington,7.0555,0.0,3.0,49.364147,12.287555
3,2020-01-01 03:00:00+00:00,Washington,6.7555,0.0,3.0,49.462437,18.864824
4,2020-01-01 04:00:00+00:00,Washington,6.4555,0.0,3.0,54.175880,17.826363


In [66]:
hourly_df.describe()

,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
count,208896.000000,208896.000000,208896.000000,208896.000000,208896.000000
mean,16.637304,0.170806,11.819407,74.786316,12.014044
std,9.089711,0.806328,20.762875,17.195688,7.273314
min,-14.194500,0.000000,0.000000,11.212664,0.000000
25%,9.650500,0.000000,0.000000,62.939872,6.952754
50%,17.000500,0.000000,3.000000,78.239647,10.446206
75%,24.105499,0.000000,3.000000,89.133457,15.315873
max,41.130501,29.000000,75.000000,100.000000,76.049515


In [67]:
hourly_df.isnull().sum()

date                    0
city                    0
temperature_2m          0
precipitation           0
weather_code            0
relative_humidity_2m    0
wind_speed_10m          0
dtype: int64

In [68]:
all_daily_data = []
for idx, response in enumerate(responses):
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}

	daily_data['city'] = city[idx]
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["precipitation_sum"] = daily_precipitation_sum

	df = pd.DataFrame(daily_data)
	all_daily_data.append(df)

daily_df = pd.concat(all_daily_data)
daily_df.head()

,date,city,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2020-01-01 00:00:00+00:00,Washington,8.0055,3.1055,0.0
1,2020-01-02 00:00:00+00:00,Washington,9.5055,-1.5445,0.0
2,2020-01-03 00:00:00+00:00,Washington,13.6055,7.6055,6.6
3,2020-01-04 00:00:00+00:00,Washington,14.6555,9.4055,10.9
4,2020-01-05 00:00:00+00:00,Washington,9.6555,1.1055,2.1


In [69]:
daily_df.describe()

,temperature_2m_max,temperature_2m_min,precipitation_sum
count,8704.000000,8704.000000,8704.000000
mean,20.990799,12.862703,4.099334
std,8.780387,8.858366,9.930643
min,-7.694500,-14.194500,0.000000
25%,13.906500,5.806500,0.000000
50%,21.405500,13.003000,0.300000
75%,28.355499,20.450500,3.500000
max,41.130501,29.580500,176.799988


In [70]:
daily_df.isnull().sum()

date                  0
city                  0
temperature_2m_max    0
temperature_2m_min    0
precipitation_sum     0
dtype: int64

In [78]:
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

hourly_df.to_sql(
    name = "weather_hourly",
    con = engine,
    if_exists = "replace",
    index = False
)

daily_df.to_sql(
    name = "weather_daily",
    con = engine,
    if_exists = "replace",
    index = False
)

print("Load completed")

Load completed
